In [4]:
import tensorflow as tf
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split

In [6]:
data = pd.read_csv(r'D:\Desktop\190305\csvfile\new\새 폴더 (3)\0D_mean_orig_channel.csv')
# x = data[data.columns[:-1]]
# y = data[data.columns[-1:]]
train, test = train_test_split(data, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train),'train samples')
print(len(val),'validation samples')
print(len(test),'test samples')
# dataset = tf.data.Dataset.from_tensor_slices((x,y))

1048 train samples
262 validation samples
328 test samples


In [19]:
train_labels = train.pop('Pr_t')
val_labels = val.pop('Pr_t')
test_labels = test.pop('Pr_t')
print(val)
print(val_labels)

      X         Y     y_plus  wedge_height  Nu_t_lsq   Alpha_t       tke  \
480   0  0.109657   19.73830             0  0.000854  0.000788  0.011838   
669   0  0.175166   70.06630             0  0.002836  0.002662  0.006715   
382   0  0.082853   33.14130             0  0.001106  0.001138  0.008983   
797   0  0.224752   40.45530             0  0.003154  0.003111  0.008225   
758   0  0.208387  122.94800             0  0.002899  0.003342  0.005850   
157   0  0.033877   13.55100             0  0.000172  0.000178  0.010383   
1543  0  0.718146  287.25900             0  0.003269  0.004157  0.002524   
1186  0  0.441538   79.47690             0  0.004791  0.004985  0.005143   
1424  0  0.618699  365.03300             0  0.003684  0.004416  0.002995   
392   0  0.084843   15.27180             0  0.000484  0.000500  0.012198   
905   0  0.278109  111.24400             0  0.003621  0.004065  0.005582   
1358  0  0.565705  333.76600             0  0.003761  0.004484  0.003319   
43    0  0.0

In [20]:
def norm(x):
    return (x - x.mean())/x.std()

def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('Pr_t')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [9]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test,shuffle=False, batch_size=batch_size)

In [10]:
for feature_batch, label_batch in train_ds.take(1):
    print('전체 특성:',list(feature_batch.keys()))
    print('나이 특성의 배치:',feature_batch['tke'])
    print('타깃의 배치:',label_batch)

전체 특성: ['Alpha_t', 'MKE_visc_diff', 'TV_conv_turb', 'dk/dy', 'dT/dx_i_abs', 'tke_diss', 'Y', 'tke_pre', 'dT/dy', 'Nu_t_lsq', 'dk/dx', 'X', 'S_12(=S_21)', 'MTV_diss', 'MTV_mol_diff', 'tke_sdm', 'dp/dx', 'S_22', 'Rp', 'U-AVG-Y', 'TV_prod(al_t_1)', 'Re', 'dv/dy', 'TV_sdm', 'MKE_pre', 'du/dy', 'S_ij_abs', 'dp/dx_i_abs', 'Rd', 'Pr', 'tke_conv_turb', 'TV_diff', 'S_11', 'Rp/Rd', 'du/dx', 'tke_diff', 'dk/dx_i_abs', 'dp/dy', 'U-AVG-Z', 'wedge_height', 'dTvar/dx_i_abs', 'tke_prod(nu_t_1_new)', 'TV', 'MKE_diss', 'U-AVG-X', 'TV_diss', 'dv/dx', 'dT/dx', 'P_AVG', 'tke', 'u_du/dx+v_dv/dx', 'y_plus']
나이 특성의 배치: tf.Tensor([0.0096368  0.00205126 0.00607366 0.00429392 0.00331897], shape=(5,), dtype=float64)
타깃의 배치: tf.Tensor([0.951553  0.857238  0.9837425 0.8376035 0.912633 ], shape=(5,), dtype=float64)


In [11]:
example_batch = next(iter(train_ds))[0]

def dump(feature_column):
    feature_layer = layers.DenseFeature(feature_column)
    print(feature_layer(example_batch).numpy())

In [12]:
batch_size = 32

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu',input_shape=[len(train.keys())]),
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
             loss='mse',
             metrics=['mse'])
model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)

Epoch 1/5
      1/Unknown - 0s 67ms/step

KeyError: 'dense_6_input'